In [ ]:
import torch
from datasets import load_dataset
from transformers import (
   AutoTokenizer,
   AutoModelForCausalLM,
   Trainer,
   TrainingArguments,
   DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
# -------------------------
# 1. Config
# -------------------------
MODEL_NAME = "EleutherAI/gpt-neo-125M"   # small free model
MAX_LEN = 256
BATCH_SIZE = 2
EPOCHS = 3
OUTPUT_DIR = "/kaggle/working/lora-medbot"
# -------------------------
# 2. Load dataset
# -------------------------
# Assume you already converted your CSV -> JSONL (train.jsonl / val.jsonl)
dataset = load_dataset("json", data_files={"train": "/kaggle/input/dataset/train.jsonl", "val": "/kaggle/input/dataset/val.jsonl"})
# Format examples into plain text
def format_example(example):
   instruction = example.get("instruction", "")
   context = example.get("input", "")
   response = example.get("output", "")
   text = f"Instruction: {instruction}\nInput: {context}\nOutput: {response}"
   return {"text": text}
dataset = dataset.map(format_example)
# -------------------------
# 3. Tokenizer
# -------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
   tokenizer.pad_token = tokenizer.eos_token  # GPT-Neo doesn’t have pad token
def tokenize_fn(example):
   return tokenizer(
       example["text"],
       truncation=True,
       padding="max_length",
       max_length=MAX_LEN,
   )
tokenized_dataset = dataset.map(tokenize_fn, batched=True, remove_columns=dataset["train"].column_names)
# -------------------------
# 4. Load base model
# -------------------------
model = AutoModelForCausalLM.from_pretrained(
   MODEL_NAME,
   torch_dtype=torch.float32,
   device_map="auto"
)
# -------------------------
# 5. LoRA config
# -------------------------
lora_config = LoraConfig(
   r=8,
   lora_alpha=16,
   target_modules=["q_proj","v_proj"],  # works for GPT-like models
   lora_dropout=0.05,
   bias="none",
   task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
# -------------------------
# 6. Trainer setup
# -------------------------
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = TrainingArguments(
   output_dir=OUTPUT_DIR,
   overwrite_output_dir=True,
   num_train_epochs=EPOCHS,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=BATCH_SIZE,
   save_steps=200,
   save_total_limit=2,
   logging_dir="./logs",
   logging_steps=50,
   eval_strategy="steps",
   eval_steps=200,
   learning_rate=2e-4,
   warmup_steps=100,
   weight_decay=0.01,
   fp16=torch.cuda.is_available(),
   push_to_hub=False,
)
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["val"],
   tokenizer=tokenizer,
   data_collator=data_collator,
)
# -------------------------
# 7. Train
# -------------------------
trainer.train()
# -------------------------
# 8. Save model
# -------------------------
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

2025-08-28 06:09:56.654893: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756361397.011340      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756361397.113274      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/231224 [00:00<?, ? examples/s]

Map:   0%|          | 0/25692 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

trainable params: 294,912 || all params: 125,493,504 || trainable%: 0.2350


/tmp/ipykernel_36/1118726800.py:89: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>